In [6]:
import numpy as np
import pandas as pd

from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import datetime
import json

<h3>1) BookList in Jupiter</h3>
<p>We need to create common class below and add subclasses</p>
<p>Here I would like to create a class for DB, base on <code>SQLAlchemestry</code></p>

In [7]:
Base = declarative_base()

class BaseNote(Base):
    # Class for notes
    __tablename__ = 'notes'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    content = Column(Text, nullable=False)
    note_type = Column(String(50), nullable=False)  # Для идентификации типа записи
    created_at = Column(DateTime, default=datetime.now)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    
    def __repr__(self):
        return f"<Note(id={self.id}, title='{self.title}', type='{self.note_type}')>"

C:\Users\urmasa\AppData\Local\Temp\ipykernel_5468\1917617995.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [8]:
# Class for Node
class Notebook:
    """Основной класс записной книжки"""
    
    def __init__(self, db_url='sqlite:///notebook.db'):
        """Инициализация записной книжки с подключением к базе данных"""
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        Session = sessionmaker(bind=self.engine)
        self.session = Session()
        self.note_classes = {}  # Регистр классов записей
    
    def register_note_type(self, note_class):
        """Регистрация нового типа записи"""
        self.note_classes[note_class.__name__] = note_class
        return note_class
    
    def add_note(self, note_instance):
        """Добавление записи в книжку"""
        if not isinstance(note_instance, BaseNote):
            raise ValueError("Запись должна быть экземпляром BaseNote")
        
        self.session.add(note_instance)
        self.session.commit()
        return note_instance
    
    def get_all_notes(self):
        """Получение всех записей"""
        return self.session.query(BaseNote).order_by(BaseNote.updated_at.desc()).all()
    
    def get_notes_by_type(self, note_type):
        """Получение записей по типу"""
        return self.session.query(BaseNote).filter(BaseNote.note_type == note_type).all()
    
    def search_notes(self, query):
        """Поиск записей по заголовку и содержанию"""
        return self.session.query(BaseNote).filter(
            (BaseNote.title.contains(query)) | (BaseNote.content.contains(query))
        ).all()
    
    def delete_note(self, note_id):
        """Удаление записи по ID"""
        note = self.session.query(BaseNote).filter(BaseNote.id == note_id).first()
        if note:
            self.session.delete(note)
            self.session.commit()
            return True
        return False
    
    def get_note_by_id(self, note_id):
        """Получение записи по ID"""
        return self.session.query(BaseNote).filter(BaseNote.id == note_id).first()
    
    def close(self):
        """Закрытие соединения с базой данных"""
        self.session.close()

<h3>2) Subclasses with different types of content</h3>

In [ ]:
class TextNote(BaseNote):
    """Класс для текстовых записей"""
    
    def __init__(self, title, content):
        super().__init__()
        self.title = title
        self.content = content
        self.note_type = 'TextNote'


In [10]:
class ContactNote(BaseNote):
    """Класс для контактов"""
    
    def __init__(self, name, phone, email=None, address=None):
        super().__init__()
        self.title = f"Контакт: {name}"
        contact_info = {
            'phone': phone,
            'email': email,
            'address': address
        }
        self.content = json.dumps(contact_info, ensure_ascii=False)
        self.note_type = 'ContactNote'
    
    def get_contact_info(self):
        """Получение информации о контакте в виде словаря"""
        return json.loads(self.content)

<h3></h3>

In [11]:
class TaskNote(BaseNote):
    """Класс для задач"""
    
    def __init__(self, title, description, due_date=None, completed=False):
        super().__init__()
        self.title = title
        task_info = {
            'description': description,
            'due_date': due_date.isoformat() if due_date else None,
            'completed': completed
        }
        self.content = json.dumps(task_info, ensure_ascii=False)
        self.note_type = 'TaskNote'
    
    def get_task_info(self):
        """Получение информации о задаче в виде словаря"""
        return json.loads(self.content)
    
    def mark_completed(self):
        """Отметить задачу как выполненную"""
        task_info = self.get_task_info()
        task_info['completed'] = True
        self.content = json.dumps(task_info, ensure_ascii=False)
        self.updated_at = datetime.now()



In [12]:
class MeetingNote(BaseNote):
    """Класс для встреч"""
    
    def __init__(self, title, participants, meeting_date, location=None):
        super().__init__()
        self.title = title
        meeting_info = {
            'participants': participants,
            'meeting_date': meeting_date.isoformat() if meeting_date else None,
            'location': location
        }
        self.content = json.dumps(meeting_info, ensure_ascii=False)
        self.note_type = 'MeetingNote'
    
    def get_meeting_info(self):
        """Получение информации о встрече в виде словаря"""
        return json.loads(self.content)

In [14]:
# Создаем экземпляр записной книжки
notebook = Notebook('sqlite:///demo_notebook.db')
    
    # Регистрируем типы записей
notebook.register_note_type(TextNote)
notebook.register_note_type(ContactNote)
notebook.register_note_type(TaskNote)
notebook.register_note_type(MeetingNote)
    
    # Добавляем различные записи
print("=== Добавление записей ===")
    
    # Текстовая запись
text_note = TextNote("Идеи для проекта", "Разработать мобильное приложение для заметок...")
notebook.add_note(text_note)
print(f"Добавлена текстовая запись: {text_note.title}")
    
    # Контакт
contact_note = ContactNote("Иван Иванов", "+7-999-123-45-67", "ivan@example.com", "Москва")
notebook.add_note(contact_note)
print(f"Добавлен контакт: {contact_note.title}")
    
    # Задача
from datetime import datetime, timedelta
due_date = datetime.now() + timedelta(days=7)
task_note = TaskNote("Завершить проект", "Доделать все основные функции", due_date)
notebook.add_note(task_note)
print(f"Добавлена задача: {task_note.title}")
    
    # Встреча
meeting_date = datetime.now() + timedelta(days=2)
meeting_note = MeetingNote("Совещание по проекту", ["Алексей", "Мария", "Петр"], meeting_date, "Конференц-зал")
notebook.add_note(meeting_note)
print(f"Добавлена встреча: {meeting_note.title}")
    
print("\n=== Все записи ===")
all_notes = notebook.get_all_notes()
for note in all_notes:
    print(f"{note.id}: {note.title} ({note.note_type}) - {note.created_at}")
    
print("\n=== Поиск записей ===")
search_results = notebook.search_notes("проект")
for note in search_results:
    print(f"Найдено: {note.title}")
    
print("\n=== Записи по типам ===")
text_notes = notebook.get_notes_by_type('TextNote')
print(f"Текстовые записи: {len(text_notes)}")
    
    # Демонстрация работы с конкретными типами записей
'''
print("\n=== Информация о контакте ===")
contact = notebook.get_notes_by_type('ContactNote')[0]
contact_info = contact.get_contact_info()
print(f"Телефон: {contact_info['phone']}")
print(f"Email: {contact_info['email']}")
'''
    # Закрываем соединение
notebook.close()

=== Добавление записей ===
Добавлена текстовая запись: Идеи для проекта
Добавлен контакт: Контакт: Иван Иванов
Добавлена задача: Завершить проект
Добавлена встреча: Совещание по проекту

=== Все записи ===
8: Совещание по проекту (MeetingNote) - 2025-10-15 11:17:41.378234
7: Завершить проект (TaskNote) - 2025-10-15 11:17:41.371088
6: Контакт: Иван Иванов (ContactNote) - 2025-10-15 11:17:41.363422
5: Идеи для проекта (TextNote) - 2025-10-15 11:17:41.355956
4: Совещание по проекту (MeetingNote) - 2025-10-15 11:17:02.205862
3: Завершить проект (TaskNote) - 2025-10-15 11:17:02.198321
2: Контакт: Иван Иванов (ContactNote) - 2025-10-15 11:17:02.191654
1: Идеи для проекта (TextNote) - 2025-10-15 11:17:02.180143

=== Поиск записей ===
Найдено: Идеи для проекта
Найдено: Завершить проект
Найдено: Совещание по проекту
Найдено: Идеи для проекта
Найдено: Завершить проект
Найдено: Совещание по проекту

=== Записи по типам ===
Текстовые записи: 2
